In [ ]:
from google.colab import drive
import os


drive.mount('/content/drive', force_remount=True)
folder_path = '/content/drive/My Drive/MSc Computer Science/Master Thesis/Annotations'

for f in os.listdir(folder_path):
    print(f)

Mounted at /content/drive
SA_Annotations_temporal-comparison.xlsx
SA_Annotations_temporal-comparison_r3.xlsx
Speaker1_annotation_r2.xlsx
Speaker1_annotation_r3.xlsx
Speaker_annotation_r4.xlsx
t_s1_annotation_r1.xlsx
t_s1_annotation_r2.xlsx
t_s2_annotations_r1.xlsx
t_s2_annotation_r2.xlsx
t_s2_annotation_r3.xlsx
t_s2_annotation_r4.xlsx
t_s2_annotation_r6.xlsx
t_s1_annotation_r4_do not use.xlsx
t_s3_annotation_r2.xlsx
t_s3_annotation_r3.xlsx
t_s4_annotation_r1.xlsx
t_s4_annotation_r2.xlsx
t_s4_annotation_r3.xlsx
t_s3_annotation_r4.xlsx
t_s3_annnotation_r1.xlsx
t_s3_annotation_r2_ny.xlsx
t_s2_annotation_r5.xlsx
t_s3_annotation_r3_ny.xlsx
t_s4_annotation_r4.xlsx
Speaker1_annotations.xlsx
t_s1_annotation_r4.xlsx
SA_Annotations_temporal-comparison_r4.xlsx
agreements_author.xlsx
agreements.xlsx
t_s3_annotation_r4_ny.xlsx


In [ ]:
class Graph:
    def __init__(self, gdict=None):
        if gdict is None:
            gdict = {}
        self.gdict = gdict
        self.edge_weights = {}  # Stores {(u1, u2): weight}

    def add_vertex(self, vertices):
        for v in vertices:
            if v not in self.gdict:
                self.gdict[v] = []

    def add_edge(self, v, neighbors, weight=None):
        for ne in neighbors:
            # Add v -> neighbor
            if v not in self.gdict:
                self.gdict[v] = []
            if ne not in self.gdict[v]:
                self.gdict[v].append(ne)

            # Add neighbor -> v (undirected)
            if ne not in self.gdict:
                self.gdict[ne] = []
            if v not in self.gdict[ne]:
                self.gdict[ne].append(v)

            # Save edge weight with sorted tuple key
            if weight is not None:
                pair = tuple(sorted([v, ne]))
                self.edge_weights[pair] = weight

    def get_edges(self):
        seen = set()
        edges = []
        for v in self.gdict:
            for ne in self.gdict[v]:
                pair = tuple(sorted([v, ne]))
                if pair not in seen:
                    seen.add(pair)
                    edges.append(pair)
        return edges

    def get_vertices(self):
        return list(self.gdict.keys())

    def get_edge_weight(self, u1, u2):
        return self.edge_weights.get(tuple(sorted([u1, u2])))

    def add_edges_from_annotation_dict(self, annotation_dict, round_number):
        if round_number in annotation_dict:
            for (u1, u2), weight in annotation_dict[round_number].items():
                self.add_edge(u1, [u2], weight)

    def apply_annotations_to_graph(graph, annotation_dict, round_num):
      for (u1, u2), score in annotation_dict.get(round_num, {}).items():
        graph.add_edge(u1, [u2], weight=score)

    def get_edges_with_weights(self):
      seen = set()
      edges_with_weights = []
      for v in self.gdict:
          for ne in self.gdict[v]:
              pair = tuple(sorted([v, ne]))
              if pair not in seen:
                  seen.add(pair)
                  weight = self.edge_weights.get(pair)
                  edges_with_weights.append((pair[0], pair[1], weight))
      return edges_with_weights


In [ ]:
def add_annotation_pair(annotation_dict, round_num, usage1, usage2, annotation):
    if round_num not in annotation_dict:
        annotation_dict[round_num] = {}

    # Ensure pair is order-independent (e.g., (u1, u2) same as (u2, u1))
    pair = tuple(sorted([usage1, usage2]))

    annotation_dict[round_num][pair] = annotation

def add_annotations_from_df_to_dict(df, annotation_dict, round_number):
    for _, row in df.iterrows():
        if row['round'] == round_number:
            u1 = row['usage_1_id']
            u2 = row['usage_2_id']
            anno = row['anno']
            add_annotation_pair(annotation_dict, round_number, u1, u2, anno)

def add_annotation_from_df_to_dict_and_graph(df, graph, annotation_dict, round_number):
    add_annotations_from_df_to_dict(df, annotation_dict, round_number)
    graph.add_edges_from_annotation_dict(annotation_dict, round_number)

In [ ]:
import pandas as pd

def compute_total_shifted_weight(graph, shift=2.5):
    total = 0
    for u, v in graph.get_edges():
        weight = graph.get_edge_weight(u, v)
        if pd.notna(weight):
            try:
                total += abs(float(weight) - shift)
            except (ValueError, TypeError):
                continue
    return total

# Diachronic

In [ ]:
filename = '/SA_Annotations_temporal-comparison_r4.xlsx'
df_d = pd.read_excel(folder_path+filename)
df_d

,round,usage_1_id,usage_1_text,usage_2_id,usage_2_text,anno,Unnamed: 6
0,1.0,u5543_tg2,NaN,u456_tg1,NaN,1,NaN
1,1.0,u571_tg1,NaN,u260_tg2,NaN,3,NaN
2,1.0,u260_tg2,NaN,u122_tg1,NaN,3,NaN
3,1.0,u122_tg1,NaN,u1728_tg1,NaN,1,NaN
4,1.0,u1728_tg1,NaN,u5543_tg2,NaN,1,NaN
...,...,...,...,...,...,...,...
150,4.0,u2006_tg2,Vi har hele tiden vært opptatt av at det er øk...,u6067_tg2,"Ja, det er vel det samme, at man kan sikkert f...",3,NaN
151,4.0,u5238_tg2,Jenters og gutters rett til utdanning er et vi...,u6033_tg2,Bærekraftig bruk av naturlige økosystemer er e...,2,NaN
152,4.0,u1126_tg1,Jeg vil også understreke at det å nå de ambisi...,u3456_tg2,"Avslutningsvis vil jeg gjerne dra fram, som je...",2,NaN
153,4.0,u456_tg1,Møtet vil sette fokus på nordisk bærekraftig u...,u571_tg1,Det skal legge opp til en økonomisk politikk s...,4,NaN


In [ ]:
clusters_dict = {}
annotation_dict = {}
usage_graph = Graph()

In [ ]:
add_annotation_from_df_to_dict_and_graph(df_d, usage_graph, annotation_dict, 1)
add_annotation_from_df_to_dict_and_graph(df_d, usage_graph, annotation_dict, 2)
add_annotation_from_df_to_dict_and_graph(df_d, usage_graph, annotation_dict, 3)
add_annotation_from_df_to_dict_and_graph(df_d, usage_graph, annotation_dict, 4)
annotation_dict

{1: {('u456_tg1', 'u5543_tg2'): 1,
  ('u260_tg2', 'u571_tg1'): 3,
  ('u122_tg1', 'u260_tg2'): 3,
  ('u122_tg1', 'u1728_tg1'): 1,
  ('u1728_tg1', 'u5543_tg2'): 1},
 2: {('u1728_tg1', 'u2619_tg1'): 2,
  ('u260_tg2', 'u2619_tg1'): 3,
  ('u102_tg1', 'u456_tg1'): 3,
  ('u102_tg1', 'u122_tg1'): 3,
  ('u1728_tg1', 'u3037_tg1'): 1,
  ('u122_tg1', 'u3037_tg1'): 2,
  ('u1126_tg1', 'u1728_tg1'): 2,
  ('u1126_tg1', 'u260_tg2'): 4,
  ('u1003_tg1', 'u456_tg1'): 4,
  ('u1003_tg1', 'u260_tg2'): 3,
  ('u1728_tg1', 'u914_tg1'): 2,
  ('u260_tg2', 'u914_tg1'): 3,
  ('u1728_tg1', 'u3016_tg1'): 1,
  ('u122_tg1', 'u3016_tg1'): 3,
  ('u1728_tg1', 'u419_tg1'): 1,
  ('u122_tg1', 'u419_tg1'): 1,
  ('u2771_tg1', 'u456_tg1'): 3,
  ('u122_tg1', 'u2771_tg1'): 3,
  ('u3033_tg1', 'u456_tg1'): 3,
  ('u3033_tg1', 'u5543_tg2'): 4,
  ('u1728_tg1', 'u2233_tg1'): 2,
  ('u122_tg1', 'u2233_tg1'): 3,
  ('u356_tg1', 'u456_tg1'): 1,
  ('u356_tg1', 'u5543_tg2'): 4,
  ('u2418_tg1', 'u456_tg1'): 2,
  ('u122_tg1', 'u2418_tg1'): 3,
 

In [ ]:
dia_loss = 9

In [ ]:
total_weight = compute_total_shifted_weight(usage_graph, shift=2.5)
print('total weight', total_weight)
normalized_loss = dia_loss / total_weight
print('normalized loss', normalized_loss)

total weight 117.0
normalized loss 0.07692307692307693


# Speaker 1

In [ ]:
import pandas as pd

filename = 't_s1_annotation_r4.xlsx'

df_s1 = pd.read_excel(folder_path + filename)
df_s1

,round,usage_1_id,usage_1_text,usage_2_id,usage_2_text,anno
0,1.0,u5543_general,NaN,u35_speaker,NaN,1.0
1,1.0,u35_speaker,NaN,u28_speaker,NaN,1.0
2,1.0,u28_speaker,NaN,u244_general,NaN,1.0
3,1.0,u244_general,NaN,u23_speaker,NaN,1.0
4,1.0,u23_speaker,NaN,u8_speaker,NaN,1.0
...,...,...,...,...,...,...
148,4.0,u1905_general,Disse er: Et grønt Norden: Det er et ønske om ...,u2006_general,Det er bred enighet om at den nåværende avtale...,3.0
149,4.0,u1143_general,"Om vi spiser mindre kjøtt, men betaler mer for...",u5865_general,Jeg er veldig stolt av å være med og sikre et ...,2.0
150,4.0,u1828_general,Stortinget satte som forutsetning for videre v...,u7_speaker,I et bærekraftperspektiv snakker man om økonom...,3.0
151,4.0,u204_general,Det skal lanseres en bærekraftig innvandringsp...,u712_general,Vi synes det er positivt at norsk industri skr...,2.0


In [ ]:
s1_usage_graph = Graph()
s1_clusters_dict = {}
s1_annotation_dict = {}

In [ ]:
add_annotation_from_df_to_dict_and_graph(df_s1, s1_usage_graph, s1_annotation_dict, 1)
add_annotation_from_df_to_dict_and_graph(df_s1, s1_usage_graph, s1_annotation_dict, 2)
add_annotation_from_df_to_dict_and_graph(df_s1, s1_usage_graph, s1_annotation_dict, 3)
add_annotation_from_df_to_dict_and_graph(df_s1, s1_usage_graph, s1_annotation_dict, 4)
# s1_annotation_dict

In [ ]:
s1_loss = 1

s1_total_weight = compute_total_shifted_weight(s1_usage_graph, shift=2.5)
print('total weight', s1_total_weight)
s1_normalized_loss = s1_loss / s1_total_weight
print('normalized loss', s1_normalized_loss)

total weight 113.5
normalized loss 0.00881057268722467


# Speaker 2

In [ ]:
import pandas as pd

filename = 't_s2_annotation_r5.xlsx'

df_s2 = pd.read_excel(folder_path + filename)
df_s2

,round,usage_1_id,usage_1_text,usage_2_id,usage_2_text,anno,Unnamed: 6
0,1.0,u1143_general,NaN,u70_speaker,NaN,3.0,NaN
1,1.0,u70_speaker,NaN,u130_speaker,NaN,4.0,NaN
2,1.0,u130_speaker,NaN,u5238_general,NaN,2.0,NaN
3,1.0,u5238_general,NaN,u2278_general,NaN,2.0,NaN
4,1.0,u2278_general,NaN,u28_speaker,NaN,3.0,NaN
...,...,...,...,...,...,...,...
134,5.0,u4931_general,Bærekraftig vekst og høyere sysselsetting er f...,u5323_general,Samtidig snakker de om miljøvern og bærekrafti...,2.0,NaN
135,5.0,u4_speaker,Slike planer og oppfølgingen av disse er med p...,u6_speaker,"Når det gjelder fiskeriforvaltningen, er bærek...",3.0,NaN
136,5.0,u1791_general,Nordisk Ministerråd har vedtatt «Ny nordisk da...,u35_speaker,Til slutt: For Kristelig Folkeparti er det pri...,2.0,NaN
137,5.0,u130_speaker,De var for så vidt enig i at vi skal forvalte ...,u4139_general,Forvaltningen av de levende marine ressursene ...,4.0,NaN


In [ ]:
s2_clusters_dict = {}
s2_annotation_dict = {}
s2_usage_graph = Graph()

In [ ]:
add_annotation_from_df_to_dict_and_graph(df_s2, s2_usage_graph, s2_annotation_dict, 1)
add_annotation_from_df_to_dict_and_graph(df_s2, s2_usage_graph, s2_annotation_dict, 2)
add_annotation_from_df_to_dict_and_graph(df_s2, s2_usage_graph, s2_annotation_dict, 3)
add_annotation_from_df_to_dict_and_graph(df_s2, s2_usage_graph, s2_annotation_dict, 4)
add_annotation_from_df_to_dict_and_graph(df_s2, s2_usage_graph, s2_annotation_dict, 5)
#s2_annotation_dict

In [ ]:
s2_loss = 5.5

s2_total_weight = compute_total_shifted_weight(s2_usage_graph, shift=2.5)
print('total weight', s2_total_weight)
s2_normalized_loss = s2_loss / s2_total_weight
print('normalized loss', s2_normalized_loss)

total weight 85.0
normalized loss 0.06470588235294118


# Speaker 3

In [ ]:
import pandas as pd

filename = 't_s3_annotation_r4.xlsx'

df_s3 = pd.read_excel(folder_path + filename)
df_s3

,round,usage_1_id,usage_1_text,usage_2_id,usage_2_text,anno,Unnamed: 6
0,1,u839_general,NaN,u6_speaker,NaN,3.0,NaN
1,1,u6_speaker,NaN,u107_speaker,NaN,3.0,NaN
2,1,u107_speaker,NaN,u912_general,NaN,2.0,NaN
3,1,u912_general,NaN,u837_general,NaN,3.0,NaN
4,1,u837_general,NaN,u163_speaker,NaN,2.0,NaN
...,...,...,...,...,...,...,...
223,4,u6_speaker,Og vi finner mange eksempler på det aller vikt...,u70_speaker,Vi ser fram til at stortingsmeldingen faktisk ...,3.0,NaN
224,4,u163_speaker,Vi må investere i bærekraftig utvikling og fje...,u1763_general,"I Bergen, hvor jeg bor, har vi selvfølgelig vå...",3.0,NaN
225,4,u108_speaker,Mitt spørsmål til finansministeren lyder: «Kap...,u154_speaker,"Vi trenger et bærekraftig samfunn, og det er e...",3.0,NaN
226,4,u3679_general,Da må vi satse på fellesskapsløsninger som sik...,u56_speaker,"I det framtidige bærekraftige Norge, som alle ...",2.0,NaN


In [ ]:
s3_clusters_dict = {}
s3_annotation_dict = {}
s3_usage_graph = Graph()

In [ ]:
add_annotation_from_df_to_dict_and_graph(df_s3, s3_usage_graph, s3_annotation_dict, 1)
add_annotation_from_df_to_dict_and_graph(df_s3, s3_usage_graph, s3_annotation_dict, 2)
add_annotation_from_df_to_dict_and_graph(df_s3, s3_usage_graph, s3_annotation_dict, 3)
add_annotation_from_df_to_dict_and_graph(df_s3, s3_usage_graph, s3_annotation_dict, 4)
#s3_annotation_dict

In [ ]:
s3_loss = 14

s3_total_weight = compute_total_shifted_weight(s3_usage_graph, shift=2.5)
print('total weight', s3_total_weight)
s3_normalized_loss = s3_loss / s3_total_weight
print('normalized loss', s3_normalized_loss)

total weight 128.5
normalized loss 0.10894941634241245


# Speaker 4

In [ ]:
import pandas as pd

filename = 't_s4_annotation_r4.xlsx'

df_s4 = pd.read_excel(folder_path + filename)
df_s4

,round,usage_1_id,usage_1_text,usage_2_id,usage_2_text,anno,Unnamed: 6
0,1.0,u8_speaker,NaN,u23_speaker,NaN,2.0,NaN
1,1.0,u23_speaker,NaN,u1535_general,NaN,3.0,NaN
2,1.0,u1535_general,NaN,u106_general,NaN,1.0,NaN
3,1.0,u106_general,NaN,u8928_general,NaN,2.0,NaN
4,1.0,u8928_general,NaN,u7359_general,NaN,1.0,NaN
...,...,...,...,...,...,...,...
159,4.0,u204_speaker,Turisme er sannsynligvis en virkelig voksende ...,u23_speaker,Regjeringen vil arbeide for sosial bærekraft g...,2.0,--
160,4.0,u143_speaker,"Det skal være bærekraftig, hvis man skal nå kl...",u1535_general,Det var noe av hele tanken bak samhandlingsref...,2.0,NaN
161,4.0,u108_speaker,"Skal vi ha bærekraftige kommuner, der fraflytt...",u1824_general,Ingen andre vil støtte bærekraftig bevegelsesf...,3.0,NaN
162,4.0,u188_speaker,Særlig er jeg opptatt av at når vi skal bygge ...,u203_speaker,Kompromisset i seg selv inneholder elementer s...,3.0,NaN


In [ ]:
s4_clusters_dict = {}
s4_annotation_dict = {}
s4_usage_graph = Graph()

In [ ]:
s4_clusters_dict = {}
s4_annotation_dict = {}
s4_usage_graph = Graph()

add_annotation_from_df_to_dict_and_graph(df_s4, s4_usage_graph, s4_annotation_dict, 1)
add_annotation_from_df_to_dict_and_graph(df_s4, s4_usage_graph, s4_annotation_dict, 2)
add_annotation_from_df_to_dict_and_graph(df_s4, s4_usage_graph, s4_annotation_dict, 3)
add_annotation_from_df_to_dict_and_graph(df_s4, s4_usage_graph, s4_annotation_dict, 4)
s4_annotation_dict

{1: {('u23_speaker', 'u8_speaker'): 2.0,
  ('u1535_general', 'u23_speaker'): 3.0,
  ('u106_general', 'u1535_general'): 1.0,
  ('u106_general', 'u8928_general'): 2.0,
  ('u7359_general', 'u8928_general'): 1.0},
 2: {('u163_speaker', 'u8928_general'): 3.0,
  ('u163_speaker', 'u7359_general'): 2.0,
  ('u28_speaker', 'u8_speaker'): 2.0,
  ('u28_speaker', 'u7359_general'): 3.0,
  ('u6_speaker', 'u8928_general'): 2.0,
  ('u6_speaker', 'u7359_general'): 2.0,
  ('u189_speaker', 'u8928_general'): 2.0,
  ('u1535_general', 'u189_speaker'): 1.0,
  ('u70_speaker', 'u8928_general'): 1.0,
  ('u1535_general', 'u70_speaker'): 1.0,
  ('u62_speaker', 'u8_speaker'): 1.0,
  ('u62_speaker', 'u7359_general'): 3.0,
  ('u57_speaker', 'u8928_general'): 2.0,
  ('u57_speaker', 'u7359_general'): 2.0,
  ('u35_speaker', 'u8928_general'): 3.0,
  ('u35_speaker', 'u7359_general'): 2.0,
  ('u188_speaker', 'u8928_general'): 3.0,
  ('u1535_general', 'u188_speaker'): 3.0,
  ('u26_speaker', 'u8928_general'): 3.0,
  ('u1535_

In [ ]:
s4_loss = 4

s4_total_weight = compute_total_shifted_weight(s4_usage_graph, shift=2.5)
print('total weight', s4_total_weight)
s4_normalized_loss = s4_loss / s4_total_weight
print('normalized loss', s4_normalized_loss)

total weight 107.0
normalized loss 0.037383177570093455
